# First model training

We retrieve the pre-trained model `YOLO11n` (yolo11n.pt) from the `YOLO` class in the `ultralytics` module. For this first training run, we only run 3 epochs to see if it works.

This first step is intended to be performed only once. The rest of the training will be carried out using the `model_training.py` script in the `scripts` directory.

In [ ]:
import os
import yaml

#PROJECT_PATH = '/content/drive/MyDrive/b2/Data/Achieving a ML Proof-of-Concept'
PROJECT_PATH = os.getcwd().split('notebooks')[0]
LABEL_PATH = os.path.join(PROJECT_PATH, 'data', 'augmented_train', 'labels')

# get the number of classes
nb_class, classes = 0, []
for filename in os.listdir(LABEL_PATH):
    with open (os.path.join(LABEL_PATH, filename), 'r') as f:
        class_ids = [line.split()[0] for line in f.readlines()]
        for class_id in class_ids:
            if class_id not in classes:
                classes.append(class_id)
                nb_class += 1

# create the dataset using yaml format (required by YOLO)
dataset_config = {
    'path': PROJECT_PATH,
    'train': 'data/augmented_train/images',
    'val': 'data/valid/images',
    'test': 'data/test/images',
    'names': {0: 'hold'}
}

CONFIG_PATH = os.path.join(PROJECT_PATH, 'data', 'augmented_train', 'data.yaml')
with open(CONFIG_PATH, 'w') as f:
    yaml.safe_dump(dataset_config, f)

In [ ]:
import torch
from ultralytics import YOLO

# load the model and set the best possible device
model = YOLO('yolo11n.pt')
device = torch.device(
    'cuda' if torch.cuda.is_available()              # cuda for NVIDIA GPUs
    else 'mps' if torch.backends.mps.is_available()  # mps for Apple silicon
    else 'cpu')                                      # cpu otherwise

print(f'Using device: {device}')

In [ ]:
# train the model
results = model.train(
    data=CONFIG_PATH,
    epochs=3,
    device=device
)

In [ ]:
# paths
TEST_DIR = os.path.join(PROJECT_PATH, 'data', 'test')
TEST_IMAGE_DIR = os.path.join(TEST_DIR, 'images')
TEST_LABEL_DIR = os.path.join(TEST_DIR, 'labels')
image_files = sorted([f for f in os.listdir(TEST_IMAGE_DIR) if f.endswith('.jpg')])

# read YOLO labels
def read_yolo_labels(label_path, img_width, img_height):
    boxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                data = line.strip().split()
                class_id = int(data[0])
                
                x_center, y_center, width, height = map(float, data[1:5]) # center_x, center_y, width, height

                # convert to pixel coordinates
                x = int((x_center - width/2) * img_width)
                y = int((y_center - height/2) * img_height)
                w = int(width * img_width)
                h = int(height * img_height)

                boxes.append((x, y, w, h))
    return boxes

In [ ]:
import cv2
import matplotlib.pyplot as plt
from datetime import datetime

fig, axes = plt.subplots(6, 2, figsize=(6.4, 4.8*4))

for ax, title in zip(axes[0], ['Predictions', 'Ground Truth']):
    ax.set_title(title, fontsize=16)

for i, img_file in enumerate(image_files[:6]):
    img_path = os.path.join(TEST_IMAGE_DIR, img_file)
    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    img_height, img_width = img.shape[:2]

    # prédictions
    axes[i, 0].imshow(img)
    axes[i, 0].axis('off')
    results = model.predict(img_path, conf=0.3)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            
            # ajoute la boîte avec score de confiance
            axes[i, 0].add_patch(plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                fill=False, edgecolor='red', linewidth=2))
            axes[i, 0].text(x1, y1-5, f'{conf:.2f}', 
                          color='red', fontsize=8, backgroundcolor='white')

    # vérité terrain
    axes[i, 1].imshow(img)
    axes[i, 1].axis('off')

    label_path = os.path.join(TEST_LABEL_DIR, f'{os.path.splitext(img_file)[0]}.txt')
    gt_boxes = read_yolo_labels(label_path, img_width, img_height)

    for x, y, w, h in gt_boxes:
        axes[i, 1].add_patch(plt.Rectangle((x, y), w, h,
                             fill=False, edgecolor='green', linewidth=2))

fig.tight_layout()
plt.show()

# save the model and the figure comparing predictions and ground truth
now = datetime.now().strftime("%Y-%m-%d_%Hh%M")
model.save(os.path.join(PROJECT_PATH, 'models', f'{now}_yolo11n.pt'))
fig.savefig(os.path.join(PROJECT_PATH, 'models', f'{now}_predictions.png'))